In [1]:
import wrds
import pandas as pd
db = wrds.Connection(wrds_username='user2294')

/Users/ishaanpoojari/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/ishaanpoojari/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


Loading library list...
Done


In [2]:
initial_holdings = ['C', 'NXTL', 'TWX', 'EIX', 'LNC', 'FE', 'ELN', 'T', 'T', 'PVN', 'IBM', 'PVN', 'COST', 'VIAV', 'PFE', 'TGNA', 'HD', 'STI', 'FCEL', 'SFE', 'MRK', 'AKAM', 'FDC', 'DD', 'CEM', 'BEV', 'GPS', 'NEOL', 'FE', 'INTC', 'WFC', 'JAVA', 'MO', 'EVRG', 'INTC', 'DD', 'DELL', 'CCI', 'FALC', 'ITWO', 'BLS', 'DOV', 'MTB', 'VIAV', 'SGP', 'DNB', 'ITWO', 'SILI', 'PARA', 'PACW', 'DD', 'TXN', 'DIS', 'ODP', 'TT', 'JNY', 'JPM', 'MCD', 'AWE', 'NHI', 'PARA', 'HRB', 'BA', 'WPO', 'PVN', 'G', 'TZH', 'CMTN', 'DNB', 'AMAT', 'OSI', 'QQQ', 'Q', 'XLNX', 'PAYX', 'ALL', 'MCO', 'TGT', 'GL', 'GPS', 'MXIM', 'CRAY', 'WM']
initial_holdings

['C',
 'NXTL',
 'TWX',
 'EIX',
 'LNC',
 'FE',
 'ELN',
 'T',
 'T',
 'PVN',
 'IBM',
 'PVN',
 'COST',
 'VIAV',
 'PFE',
 'TGNA',
 'HD',
 'STI',
 'FCEL',
 'SFE',
 'MRK',
 'AKAM',
 'FDC',
 'DD',
 'CEM',
 'BEV',
 'GPS',
 'NEOL',
 'FE',
 'INTC',
 'WFC',
 'JAVA',
 'MO',
 'EVRG',
 'INTC',
 'DD',
 'DELL',
 'CCI',
 'FALC',
 'ITWO',
 'BLS',
 'DOV',
 'MTB',
 'VIAV',
 'SGP',
 'DNB',
 'ITWO',
 'SILI',
 'PARA',
 'PACW',
 'DD',
 'TXN',
 'DIS',
 'ODP',
 'TT',
 'JNY',
 'JPM',
 'MCD',
 'AWE',
 'NHI',
 'PARA',
 'HRB',
 'BA',
 'WPO',
 'PVN',
 'G',
 'TZH',
 'CMTN',
 'DNB',
 'AMAT',
 'OSI',
 'QQQ',
 'Q',
 'XLNX',
 'PAYX',
 'ALL',
 'MCO',
 'TGT',
 'GL',
 'GPS',
 'MXIM',
 'CRAY',
 'WM']

In [3]:
def fetch_stock_data_from_wrds(tickers, start_date="2001-12-31", end_date="2005-12-31"):
    db = wrds.Connection(wrds_username='your_username')  # Connect to WRDS
    stock_data = pd.DataFrame()

    for ticker in tickers:
        # Assuming the data is coming from the 'crsp' database, and 'stock' dataset
        sql_query = f"""
        SELECT date, prc AS price
        FROM crsp.dsf
        WHERE permco IN (
            SELECT permco
            FROM crsp.dsenames
            WHERE ticker = '{ticker}' AND namedt <= '{end_date}' AND nameendt >= '{start_date}'
        )
        AND date >= '{start_date}' AND date <= '{end_date}'
        ORDER BY date
        """
        data = db.raw_sql(sql_query)
        data.set_index('date', inplace=True)
        stock_data[ticker] = data['price'].pct_change().dropna()  # Calculate returns and drop NA values

    db.close()
    return stock_data

# Example tickers
extended_tickers = ['AAPL', 'GOOG', 'MSFT', 'AMZN', 'FB', 'C', 'IBM', 'INTC', 'PFE', 'JNJ']
stock_data = fetch_stock_data_from_wrds(initial_holdings)


WRDS recommends setting up a .pgpass file.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


ValueError: cannot reindex on an axis with duplicate labels

In [ ]:
import numpy as np
from gym import Env
from gym.spaces import Box

class StockPortfolioEnv(Env):
    def __init__(self, df):
        super(StockPortfolioEnv, self).__init__()
        self.df = df
        self.n_assets = len(df.columns)
        self.action_space = Box(low=0, high=1, shape=(self.n_assets,), dtype=np.float32)
        self.observation_space = Box(low=-np.inf, high=np.inf, shape=(self.n_assets,), dtype=np.float32)
        self.initial_weights = np.zeros(self.n_assets)

    def reset(self):
        self.current_step = 0
        self.portfolio_returns = []
        self.current_weights = np.array([1.0 / self.n_assets] * self.n_assets)
        initial_obs = self.df.iloc[0].values
        return initial_obs.astype(np.float32)

    def step(self, action):
        self.current_weights = action / np.sum(action)  # Normalize the action to sum to 1
        current_returns = self.df.iloc[self.current_step].values
        portfolio_return = np.dot(current_returns, self.current_weights)
        self.portfolio_returns.append(portfolio_return)

        sharpe_ratio = self.calculate_sharpe_ratio()

        reward = sharpe_ratio
        self.current_step += 1
        done = self.current_step >= len(self.df) - 1
        next_obs = self.df.iloc[self.current_step % len(self.df)].values
        return next_obs.astype(np.float32), reward, done, {'sharpe_ratio': sharpe_ratio}

    def calculate_sharpe_ratio(self):
        if len(self.portfolio_returns) > 1:
            mean_returns = np.mean(self.portfolio_returns)
            std_returns = np.std(self.portfolio_returns)
            if std_returns > 0:
                return mean_returns / std_returns
            else:
                return 0
        return 0

    def render(self, mode='human'):
        print(f"Step: {self.current_step}, Portfolio Returns: {self.portfolio_returns[-1] if self.portfolio_returns else 'N/A'}")
        print(f'Current Weights: {self.current_weights}')
